In [1]:
!cd /content

In [2]:
from google.colab import files

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shreyanshg1101","key":"3a5df1ed9ef0ca227325d8ef2fe61af8"}'}

In [4]:
!mkdir ~/.kaggle

In [5]:
!mv ./kaggle.json ~/.kaggle

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!kaggle datasets download -d kazanova/sentiment140

 98% 79.0M/80.9M [00:03<00:00, 28.9MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.0MB/s]


In [9]:
!unzip /content/sentiment140.zip

Archive:  /content/sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [10]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 11.8 MB/s eta 0:00:00


In [11]:
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
import contractions
import re
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

In [12]:
import pandas as pd
import pickle
from keras.layers import TextVectorization
import numpy as np
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential

In [13]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [16]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [17]:
cpu_count()

2

In [18]:
data = pd.read_csv("/content/training.1600000.processed.noemoticon.csv",
                   encoding="latin-1",header=None)

In [19]:
data.shape

(1600000, 6)

In [20]:
data

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [21]:
data.rename(columns={0:"target",1:"ids",2:"date",3:"flag",4:"user",5:"text"},
            inplace=True)

In [22]:
data

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [23]:
data["text"].iloc[0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [24]:
reduced_data = data.drop(labels=data.columns[1:5],axis=1)

In [25]:
reduced_data.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [26]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, t...
1    is upset that he can't update his Facebook by ...
2    @Kenichan I dived many times for the ball. Man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [27]:
def normalize_tweet(tweet_text):

  return tweet_text.lower()

In [28]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(normalize_tweet,list(reduced_data["text"])))

In [29]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he can't update his facebook by ...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it's not behaving at all....
Name: text, dtype: object

In [30]:
def expand_contractions(tweet_text):

  return contractions.fix(tweet_text)

In [31]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(expand_contractions,list(reduced_data["text"])))

In [32]:
reduced_data["text"].head()

0    @switchfoot http://twitpic.com/2y1zl - awww, t...
1    is upset that he cannot update his facebook by...
2    @kenichan i dived many times for the ball. man...
3      my whole body feels itchy and like its on fire 
4    @nationwideclass no, it is not behaving at all...
Name: text, dtype: object

In [33]:
regex_pattern = r'@[a-zA-z0-9 ]+|#[a-zA-Z0-9 ]+|\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*|\W+|\d+|<("[^"]*"|\'[^\']*\'|[^\'">])*>|_+|[^\u0000-\u007f]+'

In [34]:
def remove_noisy_tokens(tweet_text):

  return re.sub(pattern=regex_pattern,repl=" ",string=tweet_text)

In [35]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_noisy_tokens,list(reduced_data["text"])))

In [36]:
reduced_data["text"].head()

0      twitpic com  y zl awww that is a bummer you ...
1    is upset that he cannot update his facebook by...
2          managed to save   the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4      it is not behaving at all i am mad why am i ...
Name: text, dtype: object

In [37]:
def remove_remaining_noisy_tokens(tweet_text):

  return re.sub(r'\b\w\b|[^\u0000-\u007f]+|_+|\W+',repl=" ",string=tweet_text)

In [38]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_remaining_noisy_tokens,list(reduced_data["text"])))

In [39]:
reduced_data["text"].head()

0     twitpic com   zl awww that is   bummer you sh...
1    is upset that he cannot update his facebook by...
2            managed to save the rest go out of bounds
3      my whole body feels itchy and like its on fire 
4     it is not behaving at all   am mad why am   h...
Name: text, dtype: object

In [40]:
def tokenize_tweet_text(tweet_text):
  
  return word_tokenize(tweet_text)

In [41]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(tokenize_tweet_text,list(reduced_data["text"])))

In [42]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, that, is, bummer, you...
1    [is, upset, that, he, can, not, update, his, f...
2    [managed, to, save, the, rest, go, out, of, bo...
3    [my, whole, body, feels, itchy, and, like, its...
4    [it, is, not, behaving, at, all, am, mad, why,...
Name: text, dtype: object

In [43]:
en_stop_words = list(set(stopwords.words('english')).union(set(STOP_WORDS)))

In [44]:
def is_stopword(token):
  
    return not(token in en_stop_words)

In [45]:
def remove_stopwords(tweet_text):

  return list(filter(is_stopword,tweet_text))

In [46]:
with ThreadPoolExecutor(max_workers=4) as pool:

  reduced_data["text"] = list(pool.map(remove_stopwords,list(reduced_data["text"])))

In [47]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, got,...
1    [upset, update, facebook, texting, cry, result...
2                        [managed, save, rest, bounds]
3                     [body, feels, itchy, like, fire]
4                                      [behaving, mad]
Name: text, dtype: object

In [48]:
def get_wnet_pos_tag(treebank_tag):
    if treebank_tag[1].startswith('J'):
        return (treebank_tag[0],wn.ADJ)
    elif treebank_tag[1].startswith('V'):
        return (treebank_tag[0],wn.VERB)
    elif treebank_tag[1].startswith('N'):
        return (treebank_tag[0],wn.NOUN)
    elif treebank_tag[1].startswith('R'):
        return (treebank_tag[0],wn.ADV)
    else:
        return (treebank_tag[0],wn.NOUN)

In [49]:
def get_pos_tag(tweet_text):

    return list(map(get_wnet_pos_tag,pos_tag(tweet_text)))

In [50]:
reduced_data["text"] = list(map(get_pos_tag,list(reduced_data["text"])))

In [51]:
reduced_data["text"].head()

0    [(twitpic, n), (com, n), (zl, n), (awww, n), (...
1    [(upset, a), (update, a), (facebook, n), (text...
2    [(managed, v), (save, v), (rest, n), (bounds, n)]
3    [(body, n), (feels, n), (itchy, v), (like, n),...
4                            [(behaving, v), (mad, n)]
Name: text, dtype: object

In [52]:
lemmatizer = WordNetLemmatizer()

In [53]:
def lemmatize_token(token_pos_tuple):

    if token_pos_tuple == None:
        return ""
    else:
        return lemmatizer.lemmatize(word=token_pos_tuple[0],pos=token_pos_tuple[1])

In [54]:
def lemmatize_text(tweet_text):

    if len(tweet_text) > 0:
        return list(map(lemmatize_token,tweet_text))
    else:
        return [""]

In [55]:
reduced_data["text"] = list(map(lemmatize_text,list(reduced_data["text"])))

In [56]:
reduced_data["text"].head()

0    [twitpic, com, zl, awww, bummer, shoulda, get,...
1    [upset, update, facebook, texting, cry, result...
2                          [manage, save, rest, bound]
3                      [body, feel, itchy, like, fire]
4                                        [behave, mad]
Name: text, dtype: object

In [57]:
with open("/content/tokenized_data.pkl","wb") as file_handle:

  pickle.dump(reduced_data,file_handle)

In [58]:
!mv /content/tokenized_data.pkl /content/drive/MyDrive

mv: cannot move '/content/tokenized_data.pkl' to '/content/drive/MyDrive': No such file or directory


In [59]:
!cp /content/drive/MyDrive/tokenized_data.pkl /content

cp: cannot stat '/content/drive/MyDrive/tokenized_data.pkl': No such file or directory


In [60]:
with open("/content/tokenized_data.pkl","rb") as file_handle:

  reduced_data = pickle.load(file_handle)

In [61]:
reduced_data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
1,0,"[upset, update, facebook, texting, cry, result..."
2,0,"[manage, save, rest, bound]"
3,0,"[body, feel, itchy, like, fire]"
4,0,"[behave, mad]"


In [62]:
with ThreadPoolExecutor(max_workers=4) as pool:

  max_sequence_len = max(list(pool.map(lambda x: len(x), reduced_data["text"])))

In [63]:
max_sequence_len = max(list(reduced_data["text"].apply(lambda x: len(x))))

In [64]:
max_sequence_len

50

In [65]:
reduced_rows_idx = np.argwhere(np.array(reduced_data["text"].apply(lambda x: len(x))) >= 10) 

In [66]:
reduced_rows_idx = reduced_rows_idx.reshape(reduced_rows_idx.shape[0],)

In [67]:
reduced_rows_idx

array([      0,      16,      35, ..., 1599966, 1599985, 1599988])

In [68]:
reduced_data = reduced_data.iloc[reduced_rows_idx,:]

In [69]:
reduced_data.shape

(214193, 2)

In [70]:
reduced_data.head()

,target,text
0,0,"[twitpic, com, zl, awww, bummer, shoulda, get,..."
16,0,"[hollis, death, scene, hurt, severely, watch, ..."
35,0,"[ok, sick, spend, hour, sit, shower, sick, sta..."
39,0,"[bed, class, work, gym, class, day, go, fly, m..."
44,0,"[fall, asleep, hear, tracy, girl, body, find, ..."


In [71]:
reduced_data["text"] = reduced_data["text"].apply(lambda x: " ".join(x))

In [72]:
reduced_data.head()

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
16,0,hollis death scene hurt severely watch film wr...
35,0,ok sick spend hour sit shower sick stand hold ...
39,0,bed class work gym class day go fly miss girlf...
44,0,fall asleep hear tracy girl body find sad hear...


In [73]:
filtered_reduced_data = reduced_data[reduced_data["text"] != ""]

In [74]:
filtered_reduced_data.shape

(214193, 2)

In [75]:
filtered_reduced_data.head()

,target,text
0,0,twitpic com zl awww bummer shoulda get david c...
16,0,hollis death scene hurt severely watch film wr...
35,0,ok sick spend hour sit shower sick stand hold ...
39,0,bed class work gym class day go fly miss girlf...
44,0,fall asleep hear tracy girl body find sad hear...


In [76]:
max_vocabulary_size = 30000

In [77]:
vectorization_layer = TextVectorization(max_tokens=max_vocabulary_size,
                                        output_sequence_length=max_sequence_len)

In [78]:
vectorization_layer.adapt(data=np.array(filtered_reduced_data["text"]))

In [79]:
with open("/content/token_integer_mapping.pkl","wb") as file_handle:

  pickle.dump({'config': vectorization_layer.get_config(),
             'weights': vectorization_layer.get_weights()},file_handle)

In [80]:
!mv /content/token_integer_mapping.pkl /content/drive/MyDrive

mv: cannot move '/content/token_integer_mapping.pkl' to '/content/drive/MyDrive': No such file or directory


In [81]:
!cp /content/drive/MyDrive/token_integer_mapping.pkl /content

cp: cannot stat '/content/drive/MyDrive/token_integer_mapping.pkl': No such file or directory


In [82]:
with open ("/content/token_integer_mapping.pkl","rb") as file_handle:

  vectorization_layer_attributes = pickle.load(file_handle)

In [83]:
loaded_vectorization_layer = TextVectorization.from_config(vectorization_layer_attributes["config"])

In [84]:
loaded_vectorization_layer.set_weights(vectorization_layer_attributes["weights"])

In [85]:
def vectorize_tweet(raw_tweet):

  return loaded_vectorization_layer(raw_tweet).numpy()

In [86]:
vectorized_tweets = list(filtered_reduced_data["text"].apply(vectorize_tweet))

In [87]:
with open("/content/vectorized_tweets_list.pkl","wb") as file_handle:

  pickle.dump(vectorized_tweets,file_handle)

In [88]:
!mv /content/vectorized_tweets_list.pkl /content/drive/MyDrive

mv: cannot move '/content/vectorized_tweets_list.pkl' to '/content/drive/MyDrive': No such file or directory


In [89]:
!cp /content/drive/MyDrive/vectorized_tweets_list.pkl /content

cp: cannot stat '/content/drive/MyDrive/vectorized_tweets_list.pkl': No such file or directory


In [90]:
with open("/content/vectorized_tweets_list.pkl","rb") as file_handle:

  vectorized_tweets = pickle.load(file_handle)

In [91]:
vectorized_train_tweets = vectorized_tweets[0:200000]
vectorized_cv_tweets = vectorized_tweets[200000:]

In [92]:
np.savez_compressed("./vectorized_train_tweets.npz",*vectorized_train_tweets)
np.savez_compressed("./vectorized_cv_tweets.npz",*vectorized_cv_tweets)

In [93]:
!mv /content/vectorized_train_tweets.npz /content/drive/MyDrive
!mv /content/vectorized_cv_tweets.npz /content/drive/MyDrive

mv: cannot move '/content/vectorized_train_tweets.npz' to '/content/drive/MyDrive': No such file or directory
mv: cannot move '/content/vectorized_cv_tweets.npz' to '/content/drive/MyDrive': No such file or directory


In [94]:
!cp /content/drive/MyDrive/vectorized_train_tweets.npz /content
!cp /content/drive/MyDrive/vectorized_cv_tweets.npz /content

cp: cannot stat '/content/drive/MyDrive/vectorized_train_tweets.npz': No such file or directory
cp: cannot stat '/content/drive/MyDrive/vectorized_cv_tweets.npz': No such file or directory


In [95]:
filtered_reduced_data["target"] = filtered_reduced_data["target"].apply(lambda x: str(x))

In [96]:
tweet_labels = list(filtered_reduced_data["target"].replace(to_replace=filtered_reduced_data["target"].unique(),
                                        value=list(range(len(filtered_reduced_data["target"].unique())))))

In [97]:
train_labels = tweet_labels[0:200000]
cv_labels = tweet_labels[200000:]

In [98]:
train_mb_size = 1000
num_epochs = 10
train_size = 200000

In [99]:
loaded_vectorized_train_tweets = np.load("./vectorized_train_tweets.npz")

In [100]:
def train_datagen():

  for _ in range(num_epochs):

    for i in range(train_size//train_mb_size):

      tweets_mb_list = [loaded_vectorized_train_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*train_mb_size,(i+1)*train_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in train_labels[i*train_mb_size:(i+1)*train_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [101]:
cv_mb_size = 1577
cv_size = 14193

In [102]:
loaded_vectorized_cv_tweets = np.load("./vectorized_cv_tweets.npz")

In [103]:
def cv_datagen():

  for _ in range(num_epochs):

    for i in range(cv_size//cv_mb_size):

      tweets_mb_list = [loaded_vectorized_cv_tweets["arr_"+str(arr_idx)] for arr_idx in range(i*cv_mb_size,(i+1)*cv_mb_size)]
      tweets_labels_mb_list = [np.array(tweet_label) for tweet_label in cv_labels[i*cv_mb_size:(i+1)*cv_mb_size]]

      yield np.array(tweets_mb_list), np.expand_dims(np.array(tweets_labels_mb_list),-1)

In [104]:
max_vocabulary_size = len(loaded_vectorization_layer.get_vocabulary())
embedding_dim = 64 

In [105]:
for tweets_mb_array, tweets_labels_mb_array in train_datagen():

  print(tweets_mb_array.shape)
  print(tweets_labels_mb_array.shape)

  break

(1000, 50)
(1000, 1)


In [106]:
for tweets_mb_array, tweets_labels_mb_array in cv_datagen():

  print(tweets_mb_array.shape)
  print(tweets_labels_mb_array.shape)

  break

(1577, 50)
(1577, 1)


In [107]:
def create_rnn():

  rnn_model = Sequential()

  rnn_model.add(Input(shape=(None,),dtype="int64"))
  rnn_model.add(Embedding(input_dim=30000,output_dim=64,
                          input_length=50))
  rnn_model.add(LSTM(units=64))
  rnn_model.add(Dense(units=1,activation="sigmoid"))

  return rnn_model

In [108]:
rnn_model = create_rnn()

In [109]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          1920000   
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1,953,089
Trainable params: 1,953,089
Non-trainable params: 0
_________________________________________________________________


In [110]:
import tensorflow as tf

In [111]:
rnn_model.compile(loss="binary_crossentropy",metrics=["accuracy",
                                                      tf.keras.metrics.Recall(),
                                                      tf.keras.metrics.Precision()])

In [112]:
rnn_model.fit(train_datagen(),epochs=5,validation_data=cv_datagen(),
              steps_per_epoch=100,validation_steps=9)

Epoch 1/5
100/100 [==============================] - 461s 5s/step - loss: 0.0336 - accuracy: 1.0000 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 8.1231 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/5
100/100 [==============================] - 603s 6s/step - loss: 0.3528 - accuracy: 0.8733 - recall: 0.8555 - precision: 1.0000 - val_loss: 0.0026 - val_accuracy: 1.0000 - val_recall: 1.0000 - val_precision: 1.0000
Epoch 3/5
100/100 [==============================] - 450s 5s/step - loss: 0.2294 - accuracy: 0.9200 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 7.8509 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/5
100/100 [==============================] - 634s 6s/step - loss: 0.1985 - accuracy: 0.9333 - recall: 0.9239 - precision: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000 - val_recall: 1.0000 - val_precision: 1.0000
Epoch 5/5
100/100 [==============================] - 486s 5s/ste

In [113]:
 import pickle

In [114]:
pickle.dump(rnn_model,open('model.pkl','wb'))